# Fairness under Correlation Shifts

### This Jupyter Notebook simulates the proposed pre-processing approach on the synthetic data.

We consider two scenarios: supporting (1) a single metric (DP) and (2) multiple metrics (DP & EO).

We use FairBatch [Roh et al., ICLR 2021] as an in-processing approach.

## Import libraries

In [1]:
import sys, os
import numpy as np
import math
import random 
import itertools

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
import torch

from models import LogisticRegression, weights_init_normal
from FairBatchSampler_Multiple import FairBatch, CustomDataset
from utils import correlation_reweighting, datasampling, test_model

import cvxopt
import cvxpy as cp
from cvxpy import OPTIMAL, Minimize, Problem, Variable, quad_form # Work in YJ kernel

import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir('../../')
from load_dataset import load

## Load and process the data

In the synthetic_data directory, there are a total of 6 numpy files including training data and test data.

In [3]:
from API_Design_a import MissingValueError, SamplingError, LabelError, Injector
# create pattern function given subpopulation
def create_pattern(col_list, lb_list, ub_list):
    # Check if inputs are valid
    try:
        assert len(col_list) == len(lb_list) == len(ub_list)
    except:
        print(col_list, lb_list, ub_list)
        raise SyntaxError

    def pattern(data_X, data_y):
        # Initialize a mask of all True values
        mask = np.ones(len(data_X), dtype=bool)

        # Iterate over each condition in col_list, lb_list, and ub_list
        for col, lb, ub in zip(col_list, lb_list, ub_list):
            if col == 'Y':
                mask &= (data_y >= lb) & (data_y <= ub)
            else:
                mask &= (data_X[col] >= lb) & (data_X[col] <= ub)

        # Convert Boolean mask to binary indicators (1 for True, 0 for False)
        binary_indicators = mask.astype(int)
        
        return binary_indicators

    return pattern

# lb_list = [0, 0]
# ub_list = [0, 0]
# X_train, X_test, y_train, y_test = load(dataset)
# X_train_orig, X_test_orig = X_train.copy(), X_test.copy()

# X_train_orig.reset_index(drop=True, inplace=True)
# X_test_orig.reset_index(drop=True, inplace=True)
# y_train.reset_index(drop=True, inplace=True)
# y_test.reset_index(drop=True, inplace=True)

# mv_pattern = create_pattern(['race', 'Y'], lb_list, ub_list)
# mv_pattern_len = np.sum(mv_pattern(X_train_orig, y_train))
# mv_num = min(mv_pattern_len, int(0.4*len(X_train_orig)))
# mv_err = MissingValueError(list(X_train_orig.columns).index('race'), mv_pattern, mv_num / mv_pattern_len)

# injecter = Injector(error_seq=[mv_err])
# dirty_X_train_orig, dirty_y_train, _, _ = injecter.inject(X_train_orig.copy(), y_train.copy(), 
#                                                           X_train_orig, y_train, seed=seed)

In [28]:
mv_pattern_len

1681

In [69]:
X_train, X_test, y_train, y_test = load('adult')

lb_list = [0, 6, 1]
ub_list = [0, 7, 1]
mv_pattern = create_pattern(['gender', 'workclass', 'Y'], lb_list, ub_list)
mv_pattern_len = np.sum(mv_pattern(X_train, y_train))
poi_ratio = 0.1

# lb_list = [2, 1, 1, 0]
# ub_list = [9, 1, 1, 0]
# mv_pattern = create_pattern(['education', 'gender', 'relationship', 'Y'], lb_list, ub_list)
# mv_pattern_len = np.sum(mv_pattern(X_train, y_train))
# poi_ratio = 0.2

mv_num = min(mv_pattern_len, int(poi_ratio*len(X_train)))
mv_err = SamplingError(mv_pattern, mv_num / mv_pattern_len)
injector = Injector(error_seq=[mv_err])
X_train, y_train, _, _ = injector.inject(X_train, y_train, X_train, y_train, seed=0)

y_train = y_train.replace({0: -1, 1: 1})
y_test = y_test.replace({0: -1, 1: 1})

xz_train = X_train.copy()
z_train = X_train.gender.copy()
y_noise = y_train.copy()

xz_test = X_test.copy()
z_test = X_test.gender.copy()

# y_train = y_train*2-1
# y_test = y_test*2-1

xz_train = torch.FloatTensor(xz_train.to_numpy())
y_train = torch.FloatTensor(y_train.to_numpy())
z_train = torch.FloatTensor(z_train.to_numpy())

xz_test = torch.FloatTensor(xz_test.to_numpy())
y_test = torch.FloatTensor(y_test.to_numpy())
z_test = torch.FloatTensor(z_test.to_numpy())

# os.chdir('robust_algorithms/fair-robust-selection')
# xz_train = np.load('./synthetic_data/xz_train.npy')
# y_train = np.load('./synthetic_data/y_train.npy')
# z_train = np.load('./synthetic_data/z_train.npy')
# 
# y_noise = np.load('./synthetic_data/y_noise_general.npy') # Labels with the general label flipping (details are in the paper)
# poi_ratio = 0.0
# 
# xz_test = np.load('./synthetic_data/xz_test.npy')
# y_test = np.load('./synthetic_data/y_test.npy') 
# z_test = np.load('./synthetic_data/z_test.npy')
# 
# xz_train = torch.FloatTensor(xz_train)
# y_train = torch.FloatTensor(y_train)
# z_train = torch.FloatTensor(z_train)
# 
# y_noise = torch.FloatTensor(y_noise)
# 
# xz_test = torch.FloatTensor(xz_test)
# y_test = torch.FloatTensor(y_test)
# z_test = torch.FloatTensor(z_test)
# os.chdir('../../')

In [70]:
mv_pattern_len

801

## Hyperparameters and functions

In [71]:
seeds = [0,1,2,3,4]

w = np.array([sum((z_train==1)&(y_train==1))/len(y_train), sum((z_train==0)&(y_train==1))/len(y_train), sum((z_train==1)&(y_train==-1))/len(y_train), sum((z_train==0)&(y_train==-1))/len(y_train)])
corr = 0.18
alpha = 0.005 # Used in FairBatch

In [72]:
def run_epoch(model, train_features, labels, optimizer, criterion):
    """Trains the model with the given train data.

    Args:
        model: A torch model to train.
        train_features: A torch tensor indicating the train features.
        labels: A torch tensor indicating the true labels.
        optimizer: A torch optimizer.
        criterion: A torch criterion.

    Returns:
        loss value.
    """
    
    optimizer.zero_grad()

    label_predicted = model.forward(train_features)
    loss  = criterion((F.tanh(label_predicted.squeeze())+1)/2, (labels.squeeze()+1)/2)
    loss.backward()

    optimizer.step()
    
    return loss.item()

In [73]:
def find_w_cvxpy(w, corr, gamma1, gamma2):
    
    """Solves the SDP relaxation problem.

    Args:
        w: A list indicating the original data ratio for each (y, z)-class.
        corr: A real number indicating the target correlation.
        gamma1: A real number indicating the range of Pr(y) change
        gamma2: A real number indicating the range of Pr(z) change

    Returns:
        solution for the optimization problem.
    """
    
    n = len(w)
    a = w[0]
    b = w[1]
    c = w[2]
    d = w[3]
    orig_corr = w[0]/(w[0]+w[2]) - w[1]/(w[1]+w[3])

    P0 = np.array([[1,0,0,0,-a],[0,1,0,0,-b],[0,0,1,0,-c],[0,0,0,1,-d],[-a,-b,-c,-d,0]])
    
    P1 = np.array([[0,-corr/2,0,(1-corr)/2,0],[-corr/2,0,(-1-corr)/2,0,0],[0,(-1-corr)/2,0,-corr/2,0],[(1-corr)/2,0,-corr/2,0,0],[0,0,0,0,0]])

    P2 = np.array([[0,0,0,0,1],[0,0,0,0,1],[0,0,0,0,0],[0,0,0,0,0],[1,1,0,0,0]])
    r2 = -2*(a+b)

    P3 = np.array([[0,0,0,0,1],[0,0,0,0,0],[0,0,0,0,1],[0,0,0,0,0],[1,0,1,0,0]])
    r3 = -2*(a+c)

    P4 = np.array([[0,0,0,0,1],[0,0,0,0,1],[0,0,0,0,1],[0,0,0,0,1],[1,1,1,1,0]])
    r4 = -2*1

    P5 = np.array([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,1]])

    X = cp.Variable((n+1,n+1), symmetric=True)

    constraints = [X >> 0]
    constraints = [
        cp.trace(P1 @ X) == 0,
        cp.trace(P2 @ X) + r2 <= gamma1,
        cp.trace(P2 @ X) + r2 >= -gamma1,
        cp.trace(P3 @ X) + r3 <= gamma2,
        cp.trace(P3 @ X) + r3 >= -gamma2,
        cp.trace(P4 @ X) + r4 == 0,
        cp.trace(P5 @ X) == 1,
        X >> 0
    ]
    prob = cp.Problem(cp.Minimize(cp.trace(P0 @ X)),constraints)

    result = prob.solve()

    x = X.value
    x = x[:, -1][:-1]
    return x

# 1. Supporting a single metric (DP)
### The results are in the experiments of the paper.

In [74]:
train_type = 'ours'

full_tests = []
full_trains = []

""" Find new data ratio for each (y, z)-class """  
w_new = find_w_cvxpy(w, corr, 0.1, 0.1)

""" Find example weights according to the new weight """  
our_weights = correlation_reweighting(xz_train, y_train, z_train, w, w_new)

""" Train models """
for seed in [42]:

    print("< Seed: {} >".format(seed))

    # ---------------------
    #  Initialize model, optimizer, and criterion
    # ---------------------

    useCuda = False
    if useCuda:
        model = LogisticRegression(xz_train.shape[1],1).cuda()
    else:
        model = LogisticRegression(xz_train.shape[1],1)

    torch.manual_seed(seed)
    model.apply(weights_init_normal)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999))
    criterion = torch.nn.BCELoss()

    losses = []
    
    
    # ---------------------
    #  Set data and batch sampler
    # ---------------------
    
    if train_type == 'in-processing-only':
        train_data = CustomDataset(xz_train, y_train, z_train)
    else:
        new_index = datasampling(xz_train, y_train, z_train, our_weights, seed = seed)
        train_data = CustomDataset(xz_train[new_index], y_train[new_index], z_train[new_index])

    sampler = FairBatch (model, train_data.x, train_data.y, train_data.z, batch_size = 100, alpha = alpha, target_fairness = 'eqopp', replacement = False, seed = seed)
    train_loader = torch.utils.data.DataLoader (train_data, sampler=sampler, num_workers=0)

    
    # ---------------------
    #  Model training
    # ---------------------

    for epoch in range(500):
        print(epoch, end="\r")

        tmp_loss = []

        for batch_idx, (data, target, z) in enumerate (train_loader):
            loss = run_epoch (model, data, target, optimizer, criterion)
            tmp_loss.append(loss)

        losses.append(sum(tmp_loss)/len(tmp_loss))
    
    tmp_test = test_model(model, xz_test, y_test, z_test)
    full_tests.append(tmp_test)
    
    # print("  Test accuracy: {}, Unfairness (EQOPP): {}".format(tmp_test['Acc'], tmp_test['EO_Y1_diff']))
    print("----------------------------------------------------------------------")
    

< Seed: 42 >
  Test accuracy: 0.7940239310264587, Unfairness (EQOPP): 0.18542869620068902
----------------------------------------------------------------------


In [75]:
pred_digits = model(xz_test).detach().numpy()
from sklearn.metrics import roc_auc_score
roc_auc_score((y_test + 1) / 2, 1/(1+np.exp(-pred_digits)))

0.8480511515036163

In [76]:
# measure equal opportunity, i.e. difference in true positive rates for the two groups
tpr_privileged = np.mean((pred_digits>0)[X_test.gender == 1])
tpr_unprivileged = np.mean((pred_digits>0)[X_test.gender == 0])
eq_opp = tpr_privileged - tpr_unprivileged
eq_opp

0.1743009151590087

In [1]:
import sys, os
import numpy as np
import pandas as pd
import torch
import math
import random
import itertools

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler

from argparse import Namespace
from models import LogisticRegression, weights_init_normal
from FairBatchSampler_Multiple import FairBatch, CustomDataset
from utils import correlation_reweighting, datasampling, test_model

import cvxopt
import cvxpy as cp
from cvxpy import OPTIMAL, Minimize, Problem, Variable, quad_form # Work in YJ kernel


# from aif360.algorithms.preprocessing.lfr import LFR
# from aif360.algorithms.preprocessing import Reweighing
# from aif360.datasets import BinaryLabelDataset

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression as SKLR
from tqdm import tqdm

sys.path.append(os.path.abspath("../../"))
os.chdir('../../')
from load_dataset import load

from API_Design_a import MissingValueError, SamplingError, LabelError, Injector


# create pattern function given subpopulation
def create_pattern(col_list, lb_list, ub_list):
    # Check if inputs are valid
    try:
        assert len(col_list) == len(lb_list) == len(ub_list)
    except:
        print(col_list, lb_list, ub_list)
        raise SyntaxError

    def pattern(data_X, data_y):
        # Initialize a mask of all True values
        mask = np.ones(len(data_X), dtype=bool)

        # Iterate over each condition in col_list, lb_list, and ub_list
        for col, lb, ub in zip(col_list, lb_list, ub_list):
            if col == 'Y':
                mask &= (data_y >= lb) & (data_y <= ub)
            else:
                mask &= (data_X[col] >= lb) & (data_X[col] <= ub)

        # Convert Boolean mask to binary indicators (1 for True, 0 for False)
        binary_indicators = mask.astype(int)

        return binary_indicators

    return pattern

X_train, X_test, y_train, y_test = load('adult')
# compute correlation between label and sensitive attribute using corrcoef
corr = np.corrcoef(X_train.gender, y_train)[0, 1]
# corr = 0.18

pr_y_orig = sum(y_train == 1) / len(y_train)
pr_z_orig = sum(X_train.gender == 1) / len(X_train)

lb_list = [0, 6, 1]
ub_list = [0, 7, 1]
mv_pattern = create_pattern(['gender', 'workclass', 'Y'], lb_list, ub_list)
mv_pattern_len = np.sum(mv_pattern(X_train, y_train))
poi_ratio = 0.1

mv_num = min(mv_pattern_len, int(poi_ratio*len(X_train)))
mv_err = SamplingError(mv_pattern, mv_num / mv_pattern_len)
injector = Injector(error_seq=[mv_err])
X_train, y_train, _, _ = injector.inject(X_train, y_train, X_train, y_train, seed=0)

pr_y_shifted = sum(y_train == 1) / len(y_train)
pr_z_shifted = sum(X_train.gender == 1) / len(X_train)

y_train = y_train.replace({0: -1, 1: 1})
y_test = y_test.replace({0: -1, 1: 1})

xz_train = X_train.copy()
z_train = X_train.gender.copy()
y_noise = y_train.copy()

xz_test = X_test.copy()
z_test = X_test.gender.copy()

xz_train = torch.FloatTensor(xz_train.to_numpy())
y_train = torch.FloatTensor(y_train.to_numpy())
z_train = torch.FloatTensor(z_train.to_numpy())

xz_test = torch.FloatTensor(xz_test.to_numpy())
y_test = torch.FloatTensor(y_test.to_numpy())
z_test = torch.FloatTensor(z_test.to_numpy())


print("---------- Number of Data ----------" )
print(
    "Train data : %d, Test data : %d "
    % (len(y_train), len(y_test))
)
print("------------------------------------")

w = np.array([sum((z_train==1)&(y_train==1))/len(y_train), sum((z_train==0)&(y_train==1))/len(y_train),
              sum((z_train==1)&(y_train==-1))/len(y_train), sum((z_train==0)&(y_train==-1))/len(y_train)])
alpha = 0.005 #

# test robust algo
def run_epoch(model, train_features, labels, optimizer, criterion):
    """Trains the model with the given train data.

    Args:
        model: A torch model to train.
        train_features: A torch tensor indicating the train features.
        labels: A torch tensor indicating the true labels.
        optimizer: A torch optimizer.
        criterion: A torch criterion.

    Returns:
        loss value.
    """

    optimizer.zero_grad()

    label_predicted = model.forward(train_features)
    loss = criterion((F.tanh(label_predicted.squeeze()) + 1) / 2, (labels.squeeze() + 1) / 2)
    loss.backward()

    optimizer.step()

    return loss.item()


def find_w_cvxpy(w, corr, gamma1, gamma2):
    """Solves the SDP relaxation problem.

    Args:
        w: A list indicating the original data ratio for each (y, z)-class.
        corr: A real number indicating the target correlation.
        gamma1: A real number indicating the range of Pr(y) change
        gamma2: A real number indicating the range of Pr(z) change

    Returns:
        solution for the optimization problem.
    """

    n = len(w)
    a = w[0]
    b = w[1]
    c = w[2]
    d = w[3]
    orig_corr = w[0] / (w[0] + w[2]) - w[1] / (w[1] + w[3])

    P0 = np.array([[1, 0, 0, 0, -a], [0, 1, 0, 0, -b], [0, 0, 1, 0, -c], [0, 0, 0, 1, -d], [-a, -b, -c, -d, 0]])

    P1 = np.array([[0, -corr / 2, 0, (1 - corr) / 2, 0], [-corr / 2, 0, (-1 - corr) / 2, 0, 0],
                   [0, (-1 - corr) / 2, 0, -corr / 2, 0], [(1 - corr) / 2, 0, -corr / 2, 0, 0], [0, 0, 0, 0, 0]])

    P2 = np.array([[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 1, 0, 0, 0]])
    r2 = -2 * (a + b)

    P3 = np.array([[0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [1, 0, 1, 0, 0]])
    r3 = -2 * (a + c)

    P4 = np.array([[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [1, 1, 1, 1, 0]])
    r4 = -2 * 1

    P5 = np.array([[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1]])

    X = cp.Variable((n + 1, n + 1), symmetric=True)

    constraints = [X >> 0]
    constraints = [
        cp.trace(P1 @ X) == 0,
        cp.trace(P2 @ X) + r2 <= gamma1,
        cp.trace(P2 @ X) + r2 >= -gamma1,
        cp.trace(P3 @ X) + r3 <= gamma2,
        cp.trace(P3 @ X) + r3 >= -gamma2,
        cp.trace(P4 @ X) + r4 == 0,
        cp.trace(P5 @ X) == 1,
        X >> 0
    ]
    prob = cp.Problem(cp.Minimize(cp.trace(P0 @ X)), constraints)

    result = prob.solve()

    x = X.value
    x = x[:, -1][:-1]
    return x


# Set the train data
train_data = CustomDataset(xz_train, y_noise, z_train)

seeds = [42, 43, 44, 45, 46]

fairshift_aucs = []
fairshift_eos = []

reweighing_aucs = []
reweighing_eos = []

lfr_aucs = []
lfr_eos = []

train_type = 'ours'

full_tests = []
full_trains = []

""" Find new data ratio for each (y, z)-class """
gamma_y = abs(pr_y_shifted - pr_y_orig)
gamma_z = abs(pr_z_shifted - pr_z_orig)
w_new = find_w_cvxpy(w, corr, gamma_y, gamma_z)

""" Find example weights according to the new weight """
our_weights = correlation_reweighting(xz_train, y_train, z_train, w, w_new)

print(gamma_y, gamma_z, corr)

---------- Number of Data ----------
Train data : 29361, Test data : 15060 
------------------------------------
0.020490211141231035 0.01843341145308941 0.21669868107558538


In [17]:
""" Train models """
for seed in tqdm(seeds):

    print("< Seed: {} >".format(seed))

    # ---------------------
    #  Initialize model, optimizer, and criterion
    # ---------------------

    useCuda = False
    if useCuda:
        model = LogisticRegression(xz_train.shape[1], 1).cuda()
    else:
        model = LogisticRegression(xz_train.shape[1], 1)

    torch.manual_seed(seed)
    model.apply(weights_init_normal)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999))
    criterion = torch.nn.BCELoss()

    losses = []

    # ---------------------
    #  Set data and batch sampler
    # ---------------------

    if train_type == 'in-processing-only':
        train_data = CustomDataset(xz_train, y_train, z_train)
    else:
        new_index = datasampling(xz_train, y_train, z_train, our_weights, seed=seed)
        train_data = CustomDataset(xz_train[new_index], y_train[new_index], z_train[new_index])

    sampler = FairBatch(model, train_data.x, train_data.y, train_data.z, batch_size=100, alpha=alpha,
                        target_fairness='eqopp', replacement=False, seed=seed)
    train_loader = torch.utils.data.DataLoader(train_data, sampler=sampler, num_workers=0)

    # ---------------------
    #  Model training
    # ---------------------

    for epoch in range(500):
        print(epoch, end="\r")

        tmp_loss = []

        for batch_idx, (data, target, z) in enumerate(train_loader):
            loss = run_epoch(model, data, target, optimizer, criterion)
            tmp_loss.append(loss)

        losses.append(sum(tmp_loss) / len(tmp_loss))

    pred_digits = model(xz_test).detach().numpy()
    idx_privileged = np.where((X_test.gender == 1).to_numpy() & (y_test==1).detach().numpy())[0]
    tpr_privileged = np.mean((pred_digits > 0)[idx_privileged])
    idx_protected = np.where((X_test.gender == 0).to_numpy() & (y_test==1).detach().numpy())[0]
    tpr_protected = np.mean((pred_digits > 0)[idx_protected])
    eq_opp = tpr_privileged - tpr_protected

    print("----------------------------------------------------------------------")
    print('Robust Algo:')
    print(f"Test AUC: {roc_auc_score((y_test + 1) / 2, 1/(1+np.exp(-pred_digits)))}, EO: {eq_opp}")
    print("----------------------------------------------------------------------")

    fairshift_aucs.append(roc_auc_score((y_test + 1) / 2, 1/(1+np.exp(-pred_digits))))
    fairshift_eos.append(eq_opp)


# print mean and std
print("----------------------------------------------------------------------")
print('FairShift:')
print(f"Test AUC: {np.mean(fairshift_aucs)} +- {np.std(fairshift_aucs)}")
print(f"EO: {np.mean(fairshift_eos)} +- {np.std(fairshift_eos)}")
print("----------------------------------------------------------------------")

  0%|          | 0/5 [00:00<?, ?it/s]

< Seed: 42 >


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


 20%|██        | 1/5 [00:31<02:07, 31.82s/it]

----------------------------------------------------------------------
Robust Algo:
Test AUC: 0.8482211648267988, EO: 0.21663141311432146
----------------------------------------------------------------------
< Seed: 43 >


...

 80%|████████  | 4/5 [02:08<00:32, 32.33s/it]

----------------------------------------------------------------------
Robust Algo:
Test AUC: 0.849016225732775, EO: 0.2935370899168367
----------------------------------------------------------------------
< Seed: 46 >


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


100%|██████████| 5/5 [02:39<00:00, 31.98s/it]

----------------------------------------------------------------------
Robust Algo:
Test AUC: 0.8488676008755234, EO: 0.25622068590484337
----------------------------------------------------------------------
----------------------------------------------------------------------
FairShift:
Test AUC: 0.849030985915493 +- 0.0005177387659632739
EO: 0.2209025808226237 +- 0.03789849346114616
----------------------------------------------------------------------


array([    2,     3,     5, ..., 15036, 15042, 15059])

In [27]:
from metrics import computeF1, computeFairness, computeAccuracy

In [36]:
computeF1(((y_test + 1) / 2).detach().numpy(), pred_digits>0)

0.5221990703638404

In [35]:
computeAccuracy(((y_test + 1) / 2).detach().numpy(), (pred_digits>0).astype(float).ravel())

0.802058432934927

In [34]:
(pred_digits>0).astype(float).ravel()==((y_test + 1) / 2).detach().numpy()

array([ True,  True,  True, ..., False,  True,  True])

In [29]:
(y_test + 1) / 2

tensor([0., 0., 1.,  ..., 0., 0., 1.])

In [25]:
computeFairness((pred_digits>0).astype(float), X_test, (y_test + 1) / 2, 1, 'adult')

array([-0.25622069])

In [26]:
computeFairness((pred_digits>0).astype(float), X_test, (y_test + 1) / 2, 0, 'adult')

array([-0.19342827])